In [ ]:
import snowflake as sf 
import pandas as pd
from sqlalchemy import create_engine
from pandas_profiling import ProfileReport
import warnings
warnings.filterwarnings("ignore")

### 1. Fetch data from Snowflake

In [2]:
# Establish connection
engine = create_engine(
  'snowflake://{user}:{password}@{account}/'.format(
    user = '',
    password = '',
    account = '',
  )
)

# Try connection
try:
    connection = engine.connect()
    print('Succesfuly connected to Snowflake')
except:
    print('Connection failed, check credentials')
    
# Set a cursor
con = sf.connector.connect(user = '',
                           password = '',
                           account = '',
                           warehouse='',
                           database='',
                           schema=''
                           )
sfq = con.cursor()
print('Establish connection to snowflake is succesful')


# Function to fetch results as pandas dataframe 
def fetch_results(connection, query:str):
    connection.execute(query)
    return connection.fetch_pandas_all()

Succesfuly connected to Snowflake
Establish connection to snowflake is succesful


In [6]:
# Fetch results according to query
query1 = '''SELECT DISTINCT TD.STATE, T.CLIENT_ID, MD."status" AS CLIENT_STATUS, MD.INDUSTRY AS CLIENT_INDUSTRY, T.RECEIPT_NO, TD.STATUS_CODE TRANSACTION_STATUS_CODE, TD.IS_CHARGEBACK, CD.CARD_TYPE, CD.ISSUER, CD.COUNTRY_CODE, T.READER_ID, RD.READER_TYPE, T.CREATED_AT, T.AMOUNT FROM (SELECT * FROM TRANSACTIONS WHERE CLIENT_ID IN (SELECT DISTINCT(CLIENT_ID) FROM MERCHANT_DETAIL WHERE EXTRACT(YEAR FROM ONBOARDING_DATE) = 2017 AND EXTRACT(MONTH FROM ONBOARDING_DATE) = 1) AND EXTRACT(YEAR FROM CREATED_AT) = 2017) T INNER JOIN TRANSACTION_DETAIL TD USING (RECEIPT_NO) INNER JOIN CARD_DETAIL CD ON TD.RECEIPT_NO = CD.RECEIPT_NO INNER JOIN READER_DETAIL RD ON T.READER_ID = RD.READER_ID INNER JOIN MERCHANT_DETAIL MD ON T.CLIENT_ID = MD.CLIENT_ID'''
q_1 = fetch_results(sfq, query1)
print('Retrieved data is succesful')

Retrieved data is succesful


### 2. Basic EDA

`query1` brings all the information of the transactions for the year 2017 for those users that started on January 2017. The relevant features are:

* `CLIENT_STATUS`. The actual status of the user.
* `CLIENT_INDUSTRY`. The industry to which the user belongs.
* `TRANSACTION_STATUS_CODE`. The status of the transaction where 1 indicates a valid transaction.
* `IS_CHARGEBACK`. Binary value that indicates wheter or not the transaction is a chargeback.
* `CARD_TYPE`. Type of the card, for example, debit or credit.
* `ISSUER`. The bank to which the card belongs to. 
* `READER_TYPE`. The name of the clip device used in that transaction.

In order to get a first general idea of the variables let's use the `Profile Report` function from the `pandas profiling` package, which allow us to see the distribution of each variable, missing values, interactions and correlations between them. Then, as some details seem interesting to us we're going to get deeper into each one of this details. 

In [9]:
profile = ProfileReport(q_1, title='Basic Report of variables', html={'style':{'full_width':True}})

In [10]:
profile.to_widgets()

Let's interpret one by one the output of each section. 

### 1. Overview-Warnings 

* **Missing**. Three columns `CARD_TYPE`, `ISSUER` and `COUNTRY_CODE` have less than 3% missing values. These variables come from the information of the card. `CARD_TYPE` appears to be the most relevant to take into consideration of the three to drop information because it allows us to gain insights like `it is more likely to stop using the CLIP device if the user got transactions from credit cards`.  

* **High Cardinality**. It is highly possible that we need to group countries by continents or `ISSUER` by a major category. 


### 2. Variables

We notice the following facts:

* <span style="color:blue"> Most of the transactions come from the state with code 09 and 15. </span>

* The **top client** is `7aff077912...`.

* <span style="color:blue"> Most of the users are active. </span>

* **Transactions** tend to **come from `Alimentos y Bebidas` and `Tiendas y público en general`**. It could be assumed that in general these are small and medium business because stores and restaurants like `Vips` or `Liverpool` have a terminal affiliated with a Bank. We'll further clarify this. 

* <span style="color:blue"> Most of the transactions come from `DEBIT` cards. </span> These is highly interesting because getting a credit has some requirements or limitations that debit cards do not have. For example, consider how easy is for a person to get a card issued from Oxxo or MercadoLibre.  
* The main issuers are top banks in Mexico like Bancomer, Banamex or Santander. It should be interesting to investigate what are the basic requirements for a person to get a debit card from this banks in order to further develop the inquiry of the last point. 

* <span style="color:blue"> Most of the transactions are local (Mexico). </span>

* Frequent transactions come from the `CLASSIC` device, this is the simple version of the CLIP devices. We could hypothize the following there is no competitive advantage for the user to make a trade-off between price and functionality of the device.

* <span style="color:blue"> The average ticket is nearly 1000 pesos, but the median is 200 pesos. </span> Therefore it seems that most of the users make transactions related to products or actions of first need,for example, if I go to a cornershop and I don't bring cash or I go to eat to a fonda. But we'll need to prove or reject this assumption.


#### 3. Correlations

The pearson correlation between `AMOUNT` and `IS_CHARGEBACK` is slightly positive, that is, when a chargeback occurs tends to be of a greater amount than when it doesn't occur. However, the spearman coefficient is close to cero, therefore we could not affirm that it necessarily increases the amount when a chargeback happens, further investigation is needed.  


#### 4. Missing Values

* The heatmap is showing us the **nullity correlation**. In this case, when `CARD_TYPE` is missing information, also the `ISSUER` and the `COUNTRY_CODE`. In fact, this are variables of the `CARD_DETAIL` table.

* The dendogram allow us to correlate variable completion. In this case `ISSUER` predicts the missingness of information of `COUNTRY_CODE`, when one is missing so is the other one.   